In [1]:
#load libraries
import pandas as pd
import os as os
import numpy as np
import re
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

## Crawled database from https://www.henleypassportindex.com

In [2]:
df_visa_free = pd.read_csv('henley_visa_free_11_01_2019.csv', encoding = "ISO-8859-1")

In [3]:
df_visa_free.rename(columns={'passport': 'passportfrom_alpha_2code'}, inplace=True)
df_visa_free.rename(columns={'to': 'countryto_alpha_2code'}, inplace=True)

In [4]:
df_visa_free.shape

(45173, 3)

In [5]:
df_visa_free['passportfrom_alpha_2code'].nunique()

198

In [6]:
df_visa_free['countryto_alpha_2code'].nunique()

226

In [7]:
df_visa_free.head(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree
0,AM,AF,0
1,AM,AL,1
2,AM,DZ,0
3,AM,AS,0
4,AM,AD,0


## List of world countries with ISO codes
### Source Wikipedia https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes
### Scrapped by Juanu https://www.kaggle.com/juanumusic/countries-iso-codes

In [8]:
df_iso = pd.read_csv('wikipedia-iso-country-codes.csv', encoding = "ISO-8859-1")

In [9]:
df_iso.rename(columns={'Alpha-2 code': 'country_alpha_2code'}, inplace=True)
df_iso.rename(columns={'Alpha-3 code': 'country_alpha_3code'}, inplace=True)
df_iso.rename(columns={'Numeric code': 'country_numeric_code'}, inplace=True)

In [10]:
df_iso.shape

(246, 4)

In [11]:
df_iso.sample(5)

,Country name,country_alpha_2code,country_alpha_3code,country_numeric_code
173,Philippines,PH,PHL,608
137,Martinique,MQ,MTQ,474
35,Burundi,BI,BDI,108
117,Kuwait,KW,KWT,414
193,Saudi Arabia,SA,SAU,682


## GDP per countries in USD, 2016, from the United Nations
### Source: https://datacatalog.worldbank.org/dataset/gdp-ranking

In [12]:
df_GDP_UN = pd.read_csv('UN_GDP_2016_millions.csv', encoding = "ISO-8859-1")

In [13]:
df_GDP_UN.rename(columns={'countryCode': 'country_numeric_code'}, inplace=True)
df_GDP_UN.rename(columns={'countryName': 'UN_GDP_countryName'}, inplace=True)
df_GDP_UN.rename(columns={'passport_numeric_code': 'countryto_passport_numeric_code'}, inplace=True)

In [14]:
df_GDP_UN = df_GDP_UN.drop(['UN_GDP_countryName'], axis=1)

In [15]:
df_GDP_UN['UN_GDP_currentPrices2016'] = df_GDP_UN['UN_GDP_currentPrices2016'].str.replace(',', '')
df_GDP_UN['UN_GDP_currentPrices2016'] = df_GDP_UN['UN_GDP_currentPrices2016'].astype(int)

In [16]:
df_GDP_UN.shape

(212, 2)

In [17]:
df_GDP_UN.sample(5)

,country_numeric_code,UN_GDP_currentPrices2016
128,504,103607
45,178,7778
191,780,24086
58,212,581
30,854,11695


## Population per countries in millions, 2015, from the United Nations
### Source: http://data.un.org/_Docs/SYB/CSV/SYB61_T02_Population,%20Surface%20Area%20and%20Density.csv

In [18]:
df_population_UN = pd.read_csv('UN_population_2015_millions.csv', encoding = "ISO-8859-1")

In [19]:
df_population_UN.rename(columns={'countryCode': 'country_numeric_code'}, inplace=True)
df_population_UN.rename(columns={'countryName': 'UN_population_countryName'}, inplace=True)

In [20]:
df_population_UN['UN_population_Value_2015_millions'] = df_population_UN['UN_population_Value_2015_millions'].str.replace(',', '')
df_population_UN['UN_population_Value_2015_millions'] = df_population_UN['UN_population_Value_2015_millions'].astype(float)

In [21]:
df_population_UN.shape

(228, 3)

In [22]:
df_population_UN.sample(5)

,country_numeric_code,UN_population_countryName,UN_population_Value_2015_millions
97,360,Indonesia,258.16
194,748,Eswatini,1.32
116,434,Libya,6.24
177,682,Saudi Arabia,31.56
92,336,Holy See,0.00


## Surface per countries in thousand of km2, from the United Nations
### http://data.un.org/_Docs/SYB/CSV/SYB61_T02_Population,%20Surface%20Area%20and%20Density.csv

In [23]:
df_surface_UN = pd.read_csv('UN_surface_areas.csv', encoding = "ISO-8859-1")

In [24]:
df_surface_UN.rename(columns={'countryCode': 'country_numeric_code'}, inplace=True)
df_surface_UN.rename(columns={'countryName': 'UN_surface_countryName'}, inplace=True)

In [25]:
df_surface_UN['Surface area (thousand km2)'] = df_surface_UN['Surface area (thousand km2)'].str.replace(',', '')
df_surface_UN['Surface area (thousand km2)'] = df_surface_UN['Surface area (thousand km2)'].astype(float)

In [26]:
df_surface_UN.shape

(229, 4)

In [27]:
df_surface_UN.sample(5)

,country_numeric_code,UN_surface_countryName,Surface area (thousand km2),Year_Surface area
26,72,Botswana,582.0,2015
67,233,Estonia,45.0,2015
70,238,Falkland Islands (Malvinas),12.0,2015
217,840,United States of America,9834.0,2015
59,262,Djibouti,23.0,2015


## Intersect df_visa_free and df_iso on 'country to'

In [28]:
# copy df_visa_free, rename the columns from centadata and hauntedhouse
df_visa_free2 = df_visa_free.copy()

In [29]:
df_visa_free2.rename(columns={'countryto_alpha_2code': 'country_alpha_2code'}, inplace=True)

In [30]:
# Intersect on 'countryto_alpha_2code'
df = pd.merge(df_visa_free2, df_iso, on=['country_alpha_2code'], how='left')

In [31]:
df.rename(columns={'country_alpha_2code': 'countryto_alpha_2code'}, inplace=True)
df.rename(columns={'Country name': 'countryto_Country name'}, inplace=True)
df.rename(columns={'country_alpha_3code': 'countryto_country_alpha_3code'}, inplace=True)
df.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [32]:
df.sample(10)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code
7756,TZ,DE,0,Germany,DEU,276.0
23834,KP,GB,0,United Kingdom,GBR,826.0
21001,LT,AZ,0,Azerbaijan,AZE,31.0
40821,CL,NaN,0,Namibia,NAM,516.0
6592,GB,HN,1,Honduras,HND,340.0
42565,DO,SR,1,Suriname,SUR,740.0
17626,MA,AI,0,Anguilla,AIA,660.0
9146,TR,YT,0,Mayotte,MYT,175.0
44335,CM,TC,0,Turks and Caicos,TCA,796.0
30049,KE,PL,0,Poland,POL,616.0


## Intersect df_visa_free and df_iso on 'passport'

In [33]:
df_iso.rename(columns={'country_alpha_2code': 'passportfrom_alpha_2code'}, inplace=True)

In [34]:
# Intersect on 'passportfrom_alpha_2code'
df = pd.merge(df, df_iso, on=['passportfrom_alpha_2code'], how='right')

In [35]:
df_iso.rename(columns={'passportfrom_alpha_2code': 'country_alpha_2code'}, inplace=True)

In [36]:
df.rename(columns={'Country name': 'passportfrom_Country name'}, inplace=True)

In [37]:
df = df.drop(['country_alpha_3code'], axis=1)
df = df.drop(['country_numeric_code'], axis=1)

## Intersect df and df_GDP_UN

In [38]:
df_GDP_UN2 = df_GDP_UN.copy()
df_GDP_UN2.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [39]:
df2 = pd.merge(df, df_GDP_UN2, on=['countryto_country_numeric_code'], how='right')

In [40]:
df2.rename(columns={'UN_GDP_currentPrices2016': 'countryto_UN_GDP_currentPrices2016'}, inplace=True)

In [41]:
df2.sample(10)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,passportfrom_Country name,countryto_UN_GDP_currentPrices2016
30170,GB,QA,1.0,Qatar,QAT,634.0,United Kingdom,152452
17246,ML,IR,1.0,Iran,IRN,364.0,Mali,425403
5551,TG,BN,0.0,Brunei Darussalam,BRN,96.0,Togo,11400
20340,SR,LS,0.0,Lesotho,LSO,426.0,Suriname,2241
11998,TD,ER,0.0,Eritrea,ERI,232.0,Chad,5414
38223,AF,UA,0.0,Ukraine,UKR,804.0,Afghanistan,93270
38852,TJ,US,0.0,United States Of America,USA,840.0,Tajikistan,18624475
7861,TD,CL,0.0,Chile,CHL,152.0,Chad,247046
2114,GD,AU,0.0,Australia,AUS,36.0,Grenada,1304463
8024,GD,CN,1.0,China,CHN,156.0,Grenada,11218281


## Intersect df2 and df_population_UN

In [42]:
df_population_UN2 = df_population_UN.copy()
df_population_UN2.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [43]:
df3 = pd.merge(df2, df_population_UN2, on=['countryto_country_numeric_code'], how='right')

In [44]:
df3 = df3.drop(['UN_population_countryName'], axis=1)

In [45]:
df3.rename(columns={'UN_population_Value_2015_millions': 'countryto_UN_population_Value_2015_millions'}, inplace=True)

In [46]:
df3.sample(10)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,passportfrom_Country name,countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions
18281,SV,JM,1.0,Jamaica,JAM,388.0,El Salvador,14057.0,2.87
2787,GB,BH,1.0,Bahrain,BHR,48.0,United Kingdom,32179.0,1.37
3379,US,BY,1.0,Belarus,BLR,112.0,United States Of America,47408.0,9.49
28209,TT,PS,1.0,"Palestinian Territory, Occupied",PSE,275.0,Trinidad and Tobago,13397.0,4.66
20744,SO,LI,0.0,Liechtenstein,LIE,438.0,Somalia,6194.0,0.04
11747,KE,GQ,0.0,Equatorial Guinea,GNQ,226.0,Kenya,10678.0,1.18
16545,AZ,HU,0.0,Hungary,HUN,348.0,Azerbaijan,125817.0,9.78
13223,VE,PF,1.0,French Polynesia,PYF,258.0,Venezuela,5418.0,0.28
40244,KR,ZW,1.0,Zimbabwe,ZWE,716.0,"Korea, Republic of (South Korea)",16124.0,15.78
33061,DO,SB,1.0,Solomon Islands,SLB,90.0,Dominican Republic,1134.0,0.59


## Intersect df3 and df_surface_UN

In [47]:
df_surface_UN2 = df_surface_UN.copy()
df_surface_UN2.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [48]:
df4 = pd.merge(df3, df_surface_UN2, on=['countryto_country_numeric_code'], how='right')

In [49]:
df4 = df4.drop(['UN_surface_countryName'], axis=1)

In [50]:
df4.rename(columns={'Surface area (thousand km2)': 'countryto_Surface area (thousand km2)'}, inplace=True)
df4.rename(columns={'Year_Surface area': 'countryto_Year_Surface area'}, inplace=True)

In [51]:
df4.sample(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,passportfrom_Country name,countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,countryto_Surface area (thousand km2),countryto_Year_Surface area
19546,TJ,LA,1.0,Lao People's Democratic Republic,LAO,418.0,Tajikistan,15806.0,6.66,237.0,2015
531,IS,DZ,0.0,Algeria,DZA,12.0,Iceland,159049.0,39.87,2382.0,2015
5542,AE,BN,1.0,Brunei Darussalam,BRN,96.0,United Arab Emirates,11400.0,0.42,6.0,2015
26904,MO,NE,0.0,Niger,NER,562.0,Macao,7528.0,19.90,1267.0,2015
29546,BS,PL,1.0,Poland,POL,616.0,Bahamas,471402.0,38.27,313.0,2015


## Reordering the columns

In [52]:
cols = df4.columns.tolist()
cols

['passportfrom_alpha_2code',
 'countryto_alpha_2code',
 'visafree',
 'countryto_Country name',
 'countryto_country_alpha_3code',
 'countryto_country_numeric_code',
 'passportfrom_Country name',
 'countryto_UN_GDP_currentPrices2016',
 'countryto_UN_population_Value_2015_millions',
 'countryto_Surface area (thousand km2)',
 'countryto_Year_Surface area']

In [53]:
df4 = df4[['passportfrom_alpha_2code','passportfrom_Country name','countryto_alpha_2code','countryto_Country name',
  'countryto_country_alpha_3code',
 'countryto_country_numeric_code','countryto_UN_GDP_currentPrices2016',
 'countryto_UN_population_Value_2015_millions',
 'countryto_Surface area (thousand km2)',
 'countryto_Year_Surface area','visafree']]

In [54]:
df4.head()

,passportfrom_alpha_2code,passportfrom_Country name,countryto_alpha_2code,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,countryto_Surface area (thousand km2),countryto_Year_Surface area,visafree
0,AM,Armenia,AF,Afghanistan,AFG,4.0,20235.0,33.74,653.0,2015,0.0
1,AG,Antigua and Barbuda,AF,Afghanistan,AFG,4.0,20235.0,33.74,653.0,2015,0.0
2,AO,Angola,AF,Afghanistan,AFG,4.0,20235.0,33.74,653.0,2015,0.0
3,AR,Argentina,AF,Afghanistan,AFG,4.0,20235.0,33.74,653.0,2015,0.0
4,DZ,Algeria,AF,Afghanistan,AFG,4.0,20235.0,33.74,653.0,2015,0.0


## Check data types

In [55]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40409 entries, 0 to 40408
Data columns (total 11 columns):
passportfrom_alpha_2code                       40180 non-null object
passportfrom_Country name                      40385 non-null object
countryto_alpha_2code                          40188 non-null object
countryto_Country name                         40385 non-null object
countryto_country_alpha_3code                  40385 non-null object
countryto_country_numeric_code                 40409 non-null float64
countryto_UN_GDP_currentPrices2016             40388 non-null float64
countryto_UN_population_Value_2015_millions    40408 non-null float64
countryto_Surface area (thousand km2)          40409 non-null float64
countryto_Year_Surface area                    40409 non-null int64
visafree                                       40385 non-null float64
dtypes: float64(5), int64(1), object(5)
memory usage: 3.7+ MB


## Pandas pivot_table

In [56]:
df_pivot = pd.pivot_table(df4[df4.visafree == 1],index='passportfrom_Country name',values=['countryto_UN_GDP_currentPrices2016','countryto_UN_population_Value_2015_millions','countryto_Surface area (thousand km2)','visafree'], aggfunc=np.sum, margins=False,dropna=True)
df_pivot.sort_values('visafree', ascending=False)

,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,visafree
passportfrom_Country name,,,,
Japan,86556.0,64790088.0,5922.04,171.0
"Korea, Republic of (South Korea)",100204.0,55180294.0,4825.28,170.0
Singapore,90882.0,63845672.0,4744.92,170.0
France,79589.0,50540613.0,3312.55,168.0
Germany,80899.0,49471652.0,3267.30,168.0
Sweden,79253.0,52423812.0,3336.27,167.0
Finland,79354.0,52699785.0,3340.55,167.0
Denmark,79649.0,52631388.0,3340.34,167.0
Italy,79390.0,51079375.0,3286.53,167.0


In [57]:
df_pivot

,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,visafree
passportfrom_Country name,,,,
Afghanistan,5233.0,440912.0,360.17,29.0
Albania,32007.0,23039214.0,1985.87,104.0
Algeria,14779.0,2903233.0,936.58,49.0
Andorra,73672.0,52441052.0,3130.15,149.0
Angola,15505.0,3238540.0,1105.37,48.0
Antigua and Barbuda,38165.0,26820347.0,2320.16,137.0
Argentina,66797.0,31736977.0,2957.13,155.0
Armenia,45208.0,6979278.0,1432.70,60.0
Australia,61600.0,48672020.0,2899.02,161.0


## Turn the pivot table in a Data frame

In [58]:
df_pivot_flattened = pd.DataFrame(df_pivot.to_records())
df_pivot_flattened

,passportfrom_Country name,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,visafree
0,Afghanistan,5233.0,440912.0,360.17,29.0
1,Albania,32007.0,23039214.0,1985.87,104.0
2,Algeria,14779.0,2903233.0,936.58,49.0
3,Andorra,73672.0,52441052.0,3130.15,149.0
4,Angola,15505.0,3238540.0,1105.37,48.0
5,Antigua and Barbuda,38165.0,26820347.0,2320.16,137.0
6,Argentina,66797.0,31736977.0,2957.13,155.0
7,Armenia,45208.0,6979278.0,1432.70,60.0
8,Australia,61600.0,48672020.0,2899.02,161.0
9,Austria,78493.0,51478500.0,3165.51,165.0


In [59]:
df_pivot_flattened['Surface_Rank'] = df_pivot_flattened['countryto_Surface area (thousand km2)'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['GDP_Rank'] = df_pivot_flattened['countryto_UN_GDP_currentPrices2016'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['Population_Rank'] = df_pivot_flattened['countryto_UN_population_Value_2015_millions'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['visafree_countries_Rank'] = df_pivot_flattened['visafree'].rank(method='min', ascending=False).astype(int)

In [60]:
df_pivot_flattened.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 9 columns):
passportfrom_Country name                      197 non-null object
countryto_Surface area (thousand km2)          197 non-null float64
countryto_UN_GDP_currentPrices2016             197 non-null float64
countryto_UN_population_Value_2015_millions    197 non-null float64
visafree                                       197 non-null float64
Surface_Rank                                   197 non-null int32
GDP_Rank                                       197 non-null int32
Population_Rank                                197 non-null int32
visafree_countries_Rank                        197 non-null int32
dtypes: float64(4), int32(4), object(1)
memory usage: 10.9+ KB


## Calculate the overall ranking

In [61]:
df_pivot_flattened['Overall_ranking_calculation'] = df_pivot_flattened[['visafree_countries_Rank','GDP_Rank','Surface_Rank','Population_Rank']].sum(axis=1)

In [62]:
df_pivot_flattened['New_Overall_ranking'] = df_pivot_flattened['Overall_ranking_calculation'].rank(method='min', ascending=True).astype(int)

## Gain/loss in ranking

In [63]:
df_pivot_flattened['gain-loss'] = df_pivot_flattened['visafree_countries_Rank'] - df_pivot_flattened['New_Overall_ranking']

In [64]:
df_pivot_flattened

,passportfrom_Country name,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,visafree,Surface_Rank,GDP_Rank,Population_Rank,visafree_countries_Rank,Overall_ranking_calculation,New_Overall_ranking,gain-loss
0,Afghanistan,5233.0,440912.0,360.17,29.0,197,197,195,196,785,197,-1
1,Albania,32007.0,23039214.0,1985.87,104.0,109,90,96,95,390,99,-4
2,Algeria,14779.0,2903233.0,936.58,49.0,163,171,172,166,672,172,-6
3,Andorra,73672.0,52441052.0,3130.15,149.0,30,14,40,47,131,34,13
4,Angola,15505.0,3238540.0,1105.37,48.0,157,166,162,169,654,166,3
5,Antigua and Barbuda,38165.0,26820347.0,2320.16,137.0,95,80,82,57,314,80,-23
6,Argentina,66797.0,31736977.0,2957.13,155.0,42,56,51,39,188,48,-9
7,Armenia,45208.0,6979278.0,1432.70,60.0,88,120,121,138,467,115,23
8,Australia,61600.0,48672020.0,2899.02,161.0,49,38,55,25,167,43,-18
9,Austria,78493.0,51478500.0,3165.51,165.0,18,23,33,12,86,17,-5


## Reorder the columns

In [65]:
df_pivot_flattened = df_pivot_flattened[['passportfrom_Country name',
                                         'New_Overall_ranking',
                                         'gain-loss',
                                         'visafree',
                                         'visafree_countries_Rank',
                                         'countryto_UN_GDP_currentPrices2016',
                                         'GDP_Rank',
                                         'countryto_Surface area (thousand km2)',
                                         'Surface_Rank',
                                         'countryto_UN_population_Value_2015_millions',
                                         'Population_Rank',
                                         'Overall_ranking_calculation']]

In [66]:
df_pivot_flattened.sort_values('New_Overall_ranking',ascending=True, inplace=True)
df_pivot_flattened = df_pivot_flattened.reset_index(drop=True)
df_pivot_flattened

,passportfrom_Country name,New_Overall_ranking,gain-loss,visafree,visafree_countries_Rank,countryto_UN_GDP_currentPrices2016,GDP_Rank,countryto_Surface area (thousand km2),Surface_Rank,countryto_UN_population_Value_2015_millions,Population_Rank,Overall_ranking_calculation
0,Japan,1,0,171.0,1,64790088.0,1,86556.0,4,5922.04,1,7
1,Singapore,2,0,170.0,2,63845672.0,2,90882.0,2,4744.92,3,9
2,"Korea, Republic of (South Korea)",3,-1,170.0,2,55180294.0,6,100204.0,1,4825.28,2,11
3,Denmark,4,2,167.0,6,52631388.0,9,79649.0,9,3340.34,17,41
4,Finland,5,1,167.0,6,52699785.0,7,79354.0,13,3340.55,16,42
5,Sweden,6,0,167.0,6,52423812.0,15,79253.0,14,3336.27,18,53
6,Luxembourg,7,3,166.0,10,52674381.0,8,79358.0,12,3251.89,26,56
7,San Marino,8,40,148.0,48,63583906.0,3,82694.0,7,4480.32,4,62
8,Brunei Darussalam,9,40,146.0,49,60303836.0,4,83974.0,6,4235.92,5,64
9,Italy,10,-4,167.0,6,51079375.0,28,79390.0,11,3286.53,22,67
